In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import numpy as np

In [4]:
# let's try to use this twitter dataset
# it's in data/congress-twitter
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10493874/

def load_congress_data_from_json(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    data = data[0]

    inList = data['inList']
    inWeight = data['inWeight']
    outList = data['outList']
    outWeight = data['outWeight']
    usernameList = data['usernameList']

    return inList, inWeight, outList, outWeight, usernameList

def create_congress_graph(inList, inWeight, outList, outWeight, usernameList):
    G = nx.DiGraph()
    for i, username in enumerate(usernameList):
        G.add_node(i, username=username)
    for i in range(len(inList)):
        for j, in_node in enumerate(inList[i]):
            G.add_edge(in_node, i, weight=inWeight[i][j])
        for j, out_node in enumerate(outList[i]):
            G.add_edge(i, out_node, weight=outWeight[i][j])
    return G

In [5]:
inList, inWeight, outList, outWeight, usernameList = load_congress_data_from_json('data/congress-twitter/congress_network_data.json')
G = create_congress_graph(inList, inWeight, outList, outWeight, usernameList)
print(G)

DiGraph with 475 nodes and 13289 edges


In [6]:
# sanity checks
print(G.number_of_nodes())
print(G.number_of_edges())

# from congress.edgelist,
# should be 0 4 {'weight': 0.002105263157894737}
# and 4 0 {'weight': 0.0036496350364963502}
print(G.nodes[0])
print(G.nodes[4])
print(G.get_edge_data(0, 4))
print(G.get_edge_data(4, 0))

475
13289
{'username': 'SenatorBaldwin'}
{'username': 'SenBlumenthal'}
{'weight': 0.002105263157894737}
{'weight': 0.0036496350364963502}


In [11]:
# try implementing model from the community detection paper
def community_detection(G, alpha=0.1, R=1.0, delta=0.2):
    n = len(G.nodes())
    x0 = np.random.rand(n) # generate random opinions
    print('x0 =', x0)
    x = x0.copy()
    # in the future, let's use their method of generating opinions
    rho = 1 - alpha*delta # rho is convergence rate
    A = nx.to_numpy_array(G) # adjacency matrix

    t = 0
    while True:
        t += 1
        Ni = [np.where(A[i]!=0)[0] for i in range(n)] # compute neighbor sets

        if t % 10000 == 0:
            print('t =', t)
        # update opinions 
        for i in range(n):
            if len(Ni[i]) > 0:
                x[i] = x[i] + alpha * np.mean(x[Ni[i]] - x[i])
                if t % 10000 == 0:
                    print('x[{}] = {}'.format(i, x[i]))
        # check convergence. this is wrong pretty sure.
        if np.max(np.abs(x - x0)) < 1e-5:
            print('converged at t =', t)
            break
        # update rho 
        rho *= rho
        
    # generate final communities
    communities = []
    for i in range(n):
        found = False
        for c in communities:
            if np.abs(x[i] - x[c[0]]) < R*rho**t:
                c.append(i)
                found = True
                break
        if not found:
            communities.append([i])
            
    return communities

In [12]:
subgraph = G.subgraph([0, 4, 5, 6, 7, 8, 9, 10, 11, 12])
communities = community_detection(subgraph)

x0 = [0.69497884 0.31052139 0.18576649 0.93233878 0.98597868 0.78774441
 0.46930675 0.47079206 0.22992016 0.9953653 ]
t = 10000
x[0] = 0.3864257972558971
x[1] = 0.3864257972558971
x[3] = 0.7877444148718478
x[4] = 0.5870851060638723
x[6] = 0.3864257972558971
x[7] = 0.38642579725589654
x[8] = 0.38642579725589626
x[9] = 0.38642579725589654
t = 20000
x[0] = 0.3864257972558971
x[1] = 0.3864257972558971
x[3] = 0.7877444148718478
x[4] = 0.5870851060638723
x[6] = 0.3864257972558971
x[7] = 0.38642579725589654
x[8] = 0.38642579725589626
x[9] = 0.38642579725589654
t = 30000
x[0] = 0.3864257972558971
x[1] = 0.3864257972558971
x[3] = 0.7877444148718478
x[4] = 0.5870851060638723
x[6] = 0.3864257972558971
x[7] = 0.38642579725589654
x[8] = 0.38642579725589626
x[9] = 0.38642579725589654
t = 40000
x[0] = 0.3864257972558971
x[1] = 0.3864257972558971
x[3] = 0.7877444148718478
x[4] = 0.5870851060638723
x[6] = 0.3864257972558971
x[7] = 0.38642579725589654
x[8] = 0.38642579725589626
x[9] = 0.3864257972558965